In [2]:
pip install hygese


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
import seaborn as sns
import numpy as np
import requests
from folium import FeatureGroup

import io
import os
import math
import math     
import pandas as pd 
import pytest
import matplotlib.pyplot as plt
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
import pandas as pd
import io
import requests
import folium
from folium.plugins import FastMarkerCluster
from scipy.spatial import ConvexHull
from folium import Tooltip
import numpy as np
from functools import partial
import collections
import datetime
import csv
import math
from geopy import Nominatim
from IPython.core.display import display, HTML
import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)

import datetime
R = 6371.0
EARTH_RADIUS = 6371.0
import pandas as pd
import openpyxl
from openpyxl.styles import Font, PatternFill

#calculate the average differnce between DAY AND NEW DAY
def calculate_avg_diff(dataset):
    avg_diff = 0
    for i in range(len(dataset)):
        avg_diff += abs(dataset['DAY'].iloc[i] - dataset['NEW DAY'].iloc[i])
    avg_diff /= len(dataset)
    return avg_diff


# Step 1: Calculate the average 'DAY' for each 'NEW RT' and 'NEW DAY' combination

def order_similar_to_prev(dataset):
    # Step 1: Calculate the average 'DAY' for each 'NEW RT' and 'NEW DAY' combination
    grouped_data = dataset.groupby(['NEW RT', 'NEW DAY'])['DAY'].mean().reset_index()
    grouped_data = grouped_data.rename(columns={'DAY': 'AVERAGE_DAY'})

    # Step 2: Sort these averages for each 'NEW RT' and reassign 'NEW DAY'
    grouped_data.sort_values(['NEW RT', 'AVERAGE_DAY'], inplace=True)
    grouped_data['NEW_DAY_SORTED'] = grouped_data.groupby('NEW RT').cumcount() + 1  # Start from 1

    # Merge the sorted 'NEW DAY' back to the original dataset
    dataset = dataset.merge(grouped_data[['NEW RT', 'NEW DAY', 'NEW_DAY_SORTED']], on=['NEW RT', 'NEW DAY'], how='left')
    dataset['NEW DAY'] = dataset['NEW_DAY_SORTED']
    dataset.drop('NEW_DAY_SORTED', axis=1, inplace=True)
    return dataset
    # Your 'NEW DAY' is now sorted based on the average 'DAY'
def write_ouput_files(dataset):
    # Import necessary libraries
    import pandas as pd
    import openpyxl
    from tqdm import tqdm
    from openpyxl.styles import Font, PatternFill

    # Function to auto-size columns
    def autosize_columns(worksheet):
        for column in worksheet.columns:
            max_length = 0
            for cell in column:
                try:
                    if len(str(cell.value)) > max_length:
                        max_length = len(cell.value)
                except:
                    pass
            adjusted_width = (max_length -1)
            worksheet.column_dimensions[cell.column_letter].width = adjusted_width
        return worksheet

    # Reload the original dataset
    df = dataset

    # Initialize a dictionary to hold the structured data for the route schedule
    route_schedule = {}
    # Populate the route_schedule dictionary based on the DataFrame
    for _, row in df.iterrows():
        new_rt = row['NEW RT']
        new_day = row['NEW DAY']
        city = row['CITY']
        cust_name = row['CUST NAME']
        new_stop = row['NEW STOP']
        machine = int(row['Machine'])
        old_day = row['DAY']
        comp = int(row['Companion'])
        if new_rt not in route_schedule:
            route_schedule[new_rt] = {}
        
        if new_day not in route_schedule[new_rt]:
            route_schedule[new_rt][new_day] = {'cities': set(), 'customers': []}
        
        route_schedule[new_rt][new_day]['cities'].add(city)
        route_schedule[new_rt][new_day]['customers'].append({'name': cust_name, 'stop': new_stop, 'machine': machine, 'DAY': old_day, 'NEW DAY': new_day, 'Companion': comp})
    # Check the structure by showing data for one route
    next(iter(route_schedule.items()))
    # Initialize a new Excel workbook and remove the default sheet
    wb = openpyxl.Workbook()
    default_sheet = wb.active
    wb.remove(default_sheet)

    # Function to fill in the customer details for each day and stop
    # This code fills the customer details for each day, stop, route combination.
    # It uses the route_data dictionary to find the customer details for each stop on each day.
    # If there is no data in the route_data dictionary for a given day, stop, route combination,
    # then it leaves the cell blank.

    def fill_customer_details(sheet, row, col, day, stop, route_data):
        data = route_data.get(day, {}).get('customers', [])

        for customer in data:
            if customer['stop'] == stop:
                sheet.cell(row=row, column=col, value=stop)
                sheet.cell(row=row, column=col+1, value=customer['name'].title())
                sheet.cell(row=row, column=col+2, value=f"${customer['machine']}/ ${customer['Companion']}")
                return
        sheet.cell(row=row, column=col, value="")
        sheet.cell(row=row, column=col+1, value="")
        sheet.cell(row=row, column=col+2, value="")

 
    # Font and Fill styles
    bold_font = Font(bold=True)
    italic_font = Font(italic=True)
    blue_fill = PatternFill(start_color="00CCFF", end_color="00CCFF", fill_type="solid")


    # Loop through each unique NEW RT
    for route in route_schedule.keys():
        ws = wb.create_sheet(title=str(route))
        ws.page_setup.orientation = ws.ORIENTATION_LANDSCAPE  # Set page to landscape

        # Initialize row and column pointers
        row = 1
        col = 1  
        # Initialize day counter for column headers
        day_counter = 1
        while day_counter <= 20:  # There will never be more than 25 days
            # Fill in the column headers for the days and customer details
            cell = ws.cell(row=row, column=col+1, value=f"Day {day_counter}")
            cell.font = bold_font  # Make it bold
            # Update header names and styles
            ws.cell(row=row+1, column=col, value="Stop").font = italic_font
            ws.cell(row=row+1, column=col+1, value="Customer").font = italic_font
            ws.cell(row=row+1, column=col+2, value="Sales/Comp").font = italic_font  # Changed from "Machine" to "Sales"
            # Apply light blue background to headers
            for c in range(col, col+3):
                ws.cell(row=row+1, column=c).fill = blue_fill

            # Fill in customer details for each stop (1-10)
            for stop in range(1, 9):  # There will never be more than 10 stops
                fill_customer_details(ws, row=row+1+stop, col=col, day=day_counter, stop=stop, route_data=route_schedule[route])
            
            # Update the column pointer to skip to the next day block (3 columns per day, removed empty column)
            col += 3
            day_counter += 1
            
            # Check if we've filled in 5 days, if so, skip a row and reset column pointer
            if (day_counter - 1) % 5 == 0:
                row += 11  # Skip to next block of rows (10 stops + 2 header rows + 1 empty row)
                col = 1  # Reset to column B
    
        autosize_columns(ws)
    #create workbook file name that includes the date and time
    now = datetime.datetime.now()
    date_time_str = now.strftime("%m%d%Y_%H%M%S")

    # Save the Excel workbook
    wb.save(f'./data/Route_Schedule_Formatted_{date_time_str}.xlsx')
    #save dataset to a csv file for future use
    dataset.to_csv(f'./data/reroute_dataset_{date_time_str}.csv')
def print_starts_ends(data,dataset):
    for start in data['starts']:
        crint(dataset.iloc[start]['CUST NAME'],"white", "dark grey", "italic")
    for endd in data['ends']:
        crint(dataset.iloc[endd]['CUST NAME'],"white", "dark grey", "italic")
def best_tracker(solution, prev_obj, best_obj):
    crint(f"""The current solution was completed in {solution.ObjectiveValue()} km. ""","white", "dark grey", "bold_italic")
    crint(f"""The previous best solution was completed in {best_obj} km.""","white", "dark grey", "bold_italic")
    crint(f"The most previous solution was completed in {prev_obj} km.","white", "dark grey", "bold_italic")
    if best_obj > solution.ObjectiveValue():
        best_obj = solution.ObjectiveValue()
        crint(f"Therefore, we have a new best solution with a value of {str(best_obj)} km", "red", "yellow", "bold_italic")
    else:    
        if prev_obj > solution.ObjectiveValue():
            crint(f"Therefore, we have solution that is better than the previous by {prev_obj - solution.ObjectiveValue()} km","white", "dark grey", "bold_italic")
        else:
            crint(f"Therefore, the current solution is  {solution.ObjectiveValue()-prev_obj} km longer","white", "dark grey", "bold_italic")
    prev_obj = solution.ObjectiveValue()
    return prev_obj, best_obj
def get_new_stop_count(dataset):
    # Get the number of stops from each route
    stop_count = dataset.groupby('Vehicle')['NEW DAY'].nunique()
    return stop_count
import networkx as nx
from haversine import haversine, Unit
from os import write
# Define a function to calculate the distance between two nodes
def calculate_distance(node1, node2):
    lat1, lon1 = node1['Latitude'], node1['Longitude']
    lat2, lon2 = node2['Latitude'], node2['Longitude']
    return haversine((lat1, lon1), (lat2, lon2))
def split_into_days(path, graph, min_stops, max_stops):
    days = []
    while len(path) > min_stops:
        max_distance = 0
        split_point = min(max_stops, len(path))
        for i in range(min_stops, min(max_stops, len(path) - 1)):  # ensure we don't go out of range
            distance = graph[path[i]][path[i+1]]['weight']
            if distance > max_distance:
                max_distance = distance
                split_point = i + 1
        days.append(path[:split_point])
        path = path[split_point:]
    if path:
        days.append(path)
    return days
def set_days_for_each_route(dataset, min_stops=5, max_stops=8):
    unique_routes = dataset['Vehicle'].unique()

    branch = 650
    # Iterate over unique routes
    for route in unique_routes:
        # Subset your data for the current route
        subset_data = dataset[dataset['Vehicle'] == route]
        # Create a graph for this route
        graph = nx.Graph()
        # Add nodes to the graph
        for index, row in subset_data.iterrows():
            graph.add_node(index, Latitude=row['Latitude'], Longitude=row['Longitude'])
        # Add edges to the graph (complete graph, every node is connected to every other node)
        for node1, data1 in graph.nodes(data=True):
            for node2, data2 in graph.nodes(data=True):
                if node1 != node2:
                    distance = calculate_distance(data1, data2)
                    graph.add_edge(node1, node2, weight=distance)

        
        # Find the shortest path in the graph
        # Note: This assumes that the depot is the first node in the subset_data DataFrame
        depot = subset_data.index[0]
        path = nx.approximation.christofides(graph)#, source=depot)

        # Split the path into days
        days = split_into_days(path, graph,min_stops, max_stops)
        # Assign the order of the path and the day to the 'DAY' and 'DAY_ORDER' columns of the subset_data DataFrame
        day_order = 0
    
        # print(f"Route {1+route}: \n")
        with open('prtout.txt', 'a') as f:
            f.write(f"Route {1+route}: \n")
        for day, stops in enumerate(days, start=1):
            day_order = 1
            #print(f"\n Route: {1+route} Day: {day} \n")
            for stop in stops:            
                dataset.loc[stop, 'NEW BR'] = branch
                dataset.loc[stop, 'NEW RT'] = route +1
                dataset.loc[stop, 'NEW DAY'] = day
                #subset_data.loc[stop, 'DAY ORDER'] = day_order
                dataset.loc[stop, 'DAY ORDER'] = day_order
                dataset.loc[stop, 'NEW STOP'] = day_order

                dataset.loc[dataset['CUST'] == subset_data.loc[stop, 'CUST'], 'DAY ORDER'] = day_order
                subset_data = dataset[dataset['Vehicle'] == route].copy()

                
                with open('prtout.txt', 'a') as f:
                    f.write(f"""{day_order}: {subset_data.loc[stop, 'CUST NAME']} Sales: ${int(subset_data.loc[stop, 'Machine'])} Companion: ${int(subset_data.loc[stop, 'Companion'])} \n   {subset_data.loc[stop, 'ADDRESS']} """)
                day_order += 1

        # Update the original dataset with the new 'DAY' and 'DAY_ORDER' values for the current route
        dataset.update(subset_data)

    # Save the updated dataset
    dataset.to_csv('your_updated_dataset.csv')
    return dataset
# Define a function to calculate the distance between two nodes

def set_path_for_each_route(dataset):
    # Get unique routes
    unique_routes = dataset['Vehicle'].unique()
    # Iterate over unique routes
    for route in unique_routes:
        # Subset your data for the current route
        subset_data = dataset[dataset['Vehicle'] == route]
        # Create a graph for this route
        graph = nx.Graph()
        # Add nodes to the graph
        for index, row in subset_data.iterrows():
            graph.add_node(index, Latitude=row['Latitude'], Longitude=row['Longitude'])
        # Add edges to the graph (complete graph, every node is connected to every other node)
        for node1, data1 in graph.nodes(data=True):
            for node2, data2 in graph.nodes(data=True):
                if node1 != node2:
                    distance = calculate_distance(data1, data2)
                    graph.add_edge(node1, node2, weight=distance)
        # Find the shortest path in the graph
        # Note: This assumes that the depot is the first node in the subset_data DataFrame
        depot = subset_data.index[0]
        path = nx.approximation.greedy_tsp(graph)#, source=depot)
        # Assign the order of the path to the 'DAY' column of the subset_data DataFrame
        for order, node in enumerate(path):
            subset_data.loc[node, 'DAY ORDER'] = order
        # Update the original dataset with the new 'DAY' values for the current route
        dataset.update(subset_data)
    dataset['ROUTE'] = dataset['Vehicle']
    # Save the updated dataset
    dataset.to_csv('your_updated_dataset.csv')
    return dataset
def haversinedm(lat1: float, lon1: float, lat2: float, lon2: float, unit: str = 'kilometers') -> float:
    """
    Calculate the distance between two points on the Earth's surface using the Haversine formula.

    Parameters:
        lat1 (float): Latitude of the first point in degrees.
        lon1 (float): Longitude of the first point in degrees.
        lat2 (float): Latitude of the second point in degrees.
        lon2 (float): Longitude of the second point in degrees.
        unit (str): Unit of the distance result. Default is 'kilometers'.

    Returns:
        distance (float): The distance between the two points in the specified unit.
    """
    # Check if the latitude and longitude values are within valid ranges
    if not (-90 <= lat1 <= 90) or not (-90 <= lat2 <= 90) or not (-180 <= lon1 <= 180) or not (-180 <= lon2 <= 180):
        raise ValueError("Invalid latitude or longitude values")

    # Convert the latitude and longitude values from degrees to radians
    lat1_rad = np.radians(lat1)
    lon1_rad = np.radians(lon1)
    lat2_rad = np.radians(lat2)
    lon2_rad = np.radians(lon2)

    # Calculate the differences in latitude and longitude in radians
    dlat = lat2_rad - lat1_rad
    dlon = lon2_rad - lon1_rad

    # Define helper functions to calculate the intermediate values 'a' and 'c' in the Haversine formula
    def calculate_a(dlat: float, dlon: float) -> float:
        return np.sin(dlat / 2) ** 2 + np.cos(lat1_rad) * np.cos(lat2_rad) * np.sin(dlon / 2) ** 2

    def calculate_c(a: float) -> float:
        return 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))

    # Calculate the intermediate values 'a' and 'c' using the helper functions
    a = calculate_a(dlat, dlon)
    c = calculate_c(a)

    # Calculate the distance using the Haversine formula and the Earth's radius 'R'
    distance = EARTH_RADIUS * c

    # Check if the unit is valid
    conversion_factors = {
        'miles': 0.621371,
        'kilometers': 1,
        'nautical miles': 0.539957,
    }
    if unit not in conversion_factors:
        raise ValueError("Invalid unit")

    # Convert the distance using the appropriate conversion factor based on the unit
    distance *= conversion_factors[unit]

    return distance
def calculate_a(dlat, lat1_in_radians, lat2_in_radians, dlon):
    return math.sin(dlat / 2)**2 + math.cos(lat1_in_radians) * math.cos(lat2_in_radians) * math.sin(dlon / 2)**2
def calculate_c(a):
    return 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
def print_solution(data, manager, routing, solution, dataset):
    total_distance = 0  # Total distance covered by all the vehicles
    total_load = 0  # Total load carried by all the vehicles
    total_load2 = 0  # Total load carried by all the vehicles
    previous_index = routing.Start(0)
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = 'Route  {}:\n'.format(vehicle_id)
        route_distance = 0  # Distance covered by the current vehicle
        route_load = 0  # Load carried by the current vehicle
        route_load2 = 0  # Load carried by the current vehicle
        route_count = 0  # Count of stops made by the current vehicle
        while not routing.IsEnd(index):
            plan_output += ' {} -> '.format(manager.IndexToNode(index))
            #add the index row of dataset to plan_output
            plan_output += ' {} -> '.format(dataset.iloc[index]['CUST NAME'])
            plan_output += ' {}m\n'.format(routing.GetArcCostForVehicle(previous_index, index, vehicle_id))
            route_count += 1
            node_index = manager.IndexToNode(index)  #
            route_load += data['demands'][node_index]
            route_load2 += data['demands2'][node_index]
            data['Vehicle'][node_index] = vehicle_id
            
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(previous_index, index, vehicle_id)
        route_load += data['demands'][node_index]
        route_load2 += data['demands2'][node_index]
        data['Vehicle'][node_index] = vehicle_id
        
        crint('Route {} has {} stops ({})\n'.format(vehicle_id+1, route_count, data['vehicle_capacities3'][vehicle_id]),forecolor="cyan",backcolor="navy",style="bold_italic",font_size=20)
        crint('Companion: ${:,.0f} ({}) \n'.format(route_load, data['vehicle_capacities'][vehicle_id]), indent=30,style="bold_italic")
        crint('Big 5: ${:,.0f} ({})\n'.format(route_load2, data['vehicle_capacities2'][vehicle_id]), indent=30,style="bold_italic")
        crint('Distance: {}m\n'.format(route_distance), indent=30,style="bold_italic",font_size=18)
        total_distance += route_distance
        total_load += route_load  # Add the load carried by the vehicle to the total load
        total_load2 += route_load2  # Add the load carried by the vehicle to the total load
    crint('Total Distance of all routes: {}m'.format(total_distance),forecolor="cyan",backcolor="navy",style="bold_italic",font_size=20)
    # SET dataset['NEW RT'] TO THE VALUE OF DATA['VEHICLE'] + 1 FOR ALL ROWS
    # Ensure data['Vehicle'] is a pandas Series or compatible with the dataset index
    dataset['NEW RT'] = pd.Series(data['Vehicle']) + 1

    return dataset

def create_map(data, dataset):
    """
    Generates a map with markers and polygons to visualize data points and clusters.

    Args:
        data (dict): A dictionary containing the 'Vehicle' key with a list of vehicle IDs.
        dataset (DataFrame): A pandas DataFrame containing the latitude, longitude, companion, and machine data for each data point.

    Returns:
        m (folium.Map): The generated map with markers and polygons representing the data points and clusters.
    """
    dataset_cp = dataset.copy()
    dataset_cp = dataset_cp[['Latitude', 'Longitude', 'Companion', 'Machine']]
    # add a column to dataset_cp that has vehicle from data['Vehicle']
    dataset_cp['Vehicle'] = data['Vehicle']
    companion_total = dataset_cp.groupby('Vehicle')['Companion'].sum()
    machine_total = dataset_cp.groupby('Vehicle')['Machine'].sum()
    # Calculate the size of each cluster
    cluster_sizes = dataset_cp.groupby('Vehicle').size()
    #centers_df = pd.merge(centers_df, cluster_sizes.rename('Size'), left_on='Cluster', right_index=True)
    # centers_df['Size'] = centers_df['Cluster'].apply(lambda x: len(cust_numb_dict[x]))
    m = folium.Map(tiles='https://api.mapbox.com/styles/v1/mapbox/satellite-streets-v12/tiles/256/{z}/{x}/{y}?access_token=pk.eyJ1Ijoic2FtYm9zaXMiLCJhIjoiY2xyenJyM2h5MWlvMDJpcGV3NnJ3MTdkeSJ9.AGWk1gj--GW99AM0IbF3rg', attr='Mapbox', location=[dataset['Latitude'].mean(), dataset['Longitude'].mean()], zoom_start=9, max_zoom=20)
    # attr='Mapbox',
    # name='Mapbox Satellite',
    # overlay=True,
    # control=True
    # Generate a color map for each unique vehicle
    vehicle_ids = dataset_cp['Vehicle'].unique()
    colors = [f"#{''.join([np.random.choice(list('ABCDEF0123456789')) for j in range(6)])}" for i in range(
        len(vehicle_ids))]
    color_map = dict(zip(vehicle_ids, colors))
    for vehicle in vehicle_ids:
        # Extract the data for this vehicle
        vehicle_data = dataset_cp[dataset_cp['Vehicle'] == vehicle][['Latitude', 'Longitude']].values

        # Calculate the Convex Hull of the vehicle points
        if len(vehicle_data) > 2:  # ConvexHull requires at least 3 points
            hull = ConvexHull(vehicle_data)
            # Draw the Convex Hull as a polygon on the map
            folium.Polygon(locations=vehicle_data[hull.vertices].tolist(),
                        fill=True,
                        color=color_map[vehicle],
                        tooltip=Tooltip(f"Route: {vehicle+1}, Stop Count: {cluster_sizes[vehicle]}, Total Companion: {companion_total[vehicle]}, Big 5: {machine_total[vehicle]}")).add_to(m)
        # Draw the markers for the vehicle
        FastMarkerCluster(vehicle_data.tolist()).add_to(m)
 

    # Save and render the map
    m.save('map.html')
    m.render()
    # Display the map
    return m
import folium
from matplotlib import colors as mcolors
from folium import plugins
from folium.features import DivIcon
def get_good_map(dataset):
    dataset = dataset.sort_values(['NEW RT', 'NEW DAY'])
    vehicles = dataset['NEW RT'].unique()
    # Create a Map instance
    m = folium.Map(tiles='https://api.mapbox.com/styles/v1/mapbox/satellite-streets-v12/tiles/256/{z}/{x}/{y}@2x?access_token=pk.eyJ1Ijoic2FtYm9zaXMiLCJhIjoiY2xyenJyM2h5MWlvMDJpcGV3NnJ3MTdkeSJ9.AGWk1gj--GW99AM0IbF3rg', attr='Mapbox', location=[39.7, -76.5], zoom_start=9, max_zoom=20)
    # Create a color map
    colors = list(mcolors.CSS4_COLORS.keys())  # list of named colors
    color_map = {vehicle: colors[i+13] for i, vehicle in enumerate(vehicles)}

    # Create a layer for each route
    for vehicle in dataset['NEW RT'].unique():
        route = folium.FeatureGroup(name=f'Route {vehicle}')
        vehicle_data = dataset[dataset['NEW RT'] == vehicle]
        # Add points to the route
        for index, row in vehicle_data.iterrows():
            tooltip_text = f"""
            <table style="width:100%">
                <tr>
                <th>Name:</th>
                <td><a href="https://www.google.com/search?q={row['CUST NAME']}+{row['ADDRESS']}" target="_blank">{row['CUST NAME']}</a></td>
                </tr>
                <tr>
                <th>Index:</th>
                <td>{index}</td>
                </tr>
                <tr>
                <th>Index:</th>
                <td>{index}</td>
                </tr>       
                <tr>
                <th>Address:</th>
                <td><a href="https://www.google.com/maps/dir//{row['ADDRESS']}" target="_blank">{row['ADDRESS']}</a></td>
                </tr>
                <tr>
                <th>Companion:</th>
                <td>${row['Companion']:,.2f}</td>
                </tr>
                <tr>
                <th>Sales:</th>
                <td>${row['Machine']:,.2f}0</td>
                </tr>
                <tr>
                <th>Day:</th>
                <td>{int(row['NEW DAY'])}</td>
                </tr>
            </table>
            """
            iframe = folium.IFrame(html=tooltip_text, width=400, height=150)
            folium.CircleMarker(
                location=[row['Latitude'], row['Longitude']],
                popup=tooltip_text,
                tooltip=tooltip_text,
                color='#3186cc',
                fill=True,
                fill_color='#3186cc',
                radius = 12,
                weight = 5
            ).add_to(route)
        # Add lines to the route
        locations = vehicle_data[['Latitude', 'Longitude']].values.tolist()
        folium.Polygon(locations=locations, color=color_map[vehicle], fill=True, fill_opacity=0.3, weight=2).add_to(route)
        # Add the route to the map
        route.add_to(m)
    plugins.Fullscreen(
    position="topleft",
    title="Expand me",
    title_cancel="Exit me",
    force_separate_button=True,
    ).add_to(m)
    # Add a layer control to the map
    folium.LayerControl().add_to(m)
    m.save("goodmap.html")
    return m
def crint(string_to_print, forecolor='black', backcolor='white', style='normal', indent=0, font_size=17):
    """
    Custom print function to display colored text in a Jupyter Notebook using HTML.
    Parameters:
        string_to_print (str): The string to print.
        forecolor (str): The text color.
        backcolor (str): The background color.
        style (str): The text style ('normal', 'bold', 'italic').
        indent (int): The left margin indentation in pixels.
    """
    style_mapping = {
        'normal': 'font-style: normal; font-weight: normal; text-decoration: none;',
        'bold': 'font-style: normal; font-weight: bold; text-decoration: none;',
        'italic': 'font-style: italic; font-weight: normal; text-decoration: none;',
        'bold_italic': 'font-style: italic; font-weight: bold; text-decoration: none;',
        'underline': 'font-style: normal; font-weight: normal; text-decoration: underline;',
        'strikethrough': 'font-style: normal; font-weight: normal; text-decoration: line-through;',
        'overline': 'font-style: normal; font-weight: normal; text-decoration: overline;'
    }

    
    css_style = f"color: {forecolor}; background-color: {backcolor}; {style_mapping.get(style, 'font-style: normal; font-weight: normal; text-decoration: none;')} margin-left: {indent}px; font-size: {font_size}px"
    display(HTML(f"<span style='{css_style}'>{string_to_print}</span>"))
#imports to perform a wait
import time 
import sys
from IPython.display import clear_output
def print_days_per_route(dataset, data):
    i = 0
    for cunt in dataset.groupby('NEW RT')['NEW DAY'].nunique():
        i += 1
        crint(f"Route {i} Days: {cunt} ({data['demands3'][i]})",'cyan', 'navy')
def write_solution(dataset):
    now = datetime.datetime.now()
    with open(f'routewithdays {now.day}-{now.month}-{now.year} {now.hour}{now.minute}.csv','a') as ff:
        writer = csv.writer(ff)
        writer.writerow(["Route", "Route Day", "Stop", "Name", "Address", "Big 5", "Companion", "Distance"])
        for vehicle_id in dataset['Vehicle'].unique():
            c_stop = 0
            # Loop through all rows in dataset where dataset['Vehicle'] == vehicle_id
            subset_data = dataset[dataset['Vehicle'] == vehicle_id]
            for index, row in subset_data.iterrows():
                c_route = vehicle_id + 1
                c_stop += 1
                c_name = row['CUST NAME']
                c_addy = row['ADDRESS']
                c_big = row['Machine']
                c_comp = row['Companion']
                c_route_day = row['DAY']
                writer.writerow([c_route, c_route_day, c_stop, c_name, c_addy, c_big, c_comp])

def create_clean_map(data, dataset):
    """
    Generates a map with markers and polygons to visualize data points and clusters in a cleaner, more efficient way.

    Args:
        data (dict): A dictionary containing the 'Vehicle' key with a list of vehicle IDs.
        dataset (DataFrame): A DataFrame containing latitude, longitude, companion, and machine data for each data point.

    Returns:
        folium.Map: The generated map with markers and polygons representing the data points and clusters.
    """
    # Ensure 'Vehicle' data is in a format that can be used to create a new column in dataset
    if 'Vehicle' in data:
        dataset['Vehicle'] = pd.Categorical(dataset.index.map(lambda x: data['Vehicle'][x % len(data['Vehicle'])]))
    else:
        raise ValueError("Data must contain 'Vehicle' key with a list of vehicle IDs.")
    
    # Subset the dataset to relevant columns
    relevant_columns = ['Latitude', 'Longitude', 'Companion', 'Machine', 'Vehicle']
    dataset = dataset[relevant_columns].dropna(subset=['Latitude', 'Longitude'])

    # Aggregate data
    companion_total = dataset.groupby('Vehicle')['Companion'].sum()
    machine_total = dataset.groupby('Vehicle')['Machine'].sum()
    cluster_sizes = dataset.groupby('Vehicle').size()

    # Initialize map
    map_center = [dataset['Latitude'].mean(), dataset['Longitude'].mean()]
    m = folium.Map(location=map_center, zoom_start=9, max_zoom=20, tiles='https://api.mapbox.com/styles/v1/mapbox/satellite-streets-v12/tiles/256/{z}/{x}/{y}?access_token=pk.eyJ1Ijoic2FtYm9zaXMiLCJhIjoiY2xyenJyM2h5MWlvMDJpcGV3NnJ3MTdkeSJ9.AGWk1gj--GW99AM0IbF3rg', attr='Mapbox')

    # Generate color map
    vehicle_ids = dataset['Vehicle'].unique()
    colors = [f"#{''.join(np.random.choice(list('ABCDEF0123456789'), 6))}" for _ in vehicle_ids]
    color_map = dict(zip(vehicle_ids, colors))

    for vehicle in vehicle_ids:
        vehicle_data = dataset[dataset['Vehicle'] == vehicle][['Latitude', 'Longitude']]
        if len(vehicle_data) > 2:
            points = vehicle_data.values
            try:
                hull = ConvexHull(points)
                folium.Polygon(locations=points[hull.vertices].tolist(), fill=True, color=color_map[vehicle], tooltip=Tooltip(f"Vehicle: {vehicle}, Stop Count: {cluster_sizes[vehicle]}, Total Companion: {companion_total[vehicle]}, Total Machine: {machine_total[vehicle]}")).add_to(m)
            except Exception as e:
                print(f"Could not create a convex hull for vehicle {vehicle}: {e}")
        FastMarkerCluster(data=vehicle_data.values.tolist()).add_to(m)

    return m

# Example usage (assuming 'data' and 'dataset' are predefined and properly formatted)
# m = create_clean_map(data, dataset)
# m.save('clean_map.html')

import matplotlib.colors as mcolors

def create_clean_map_from_dataset(dataset):
    """
    Generates a map with markers and polygons to visualize routes based on the 'NEW RT' column in the dataset.

    Args:
        dataset (DataFrame): A pandas DataFrame containing latitude, longitude, and other data for each point.

    Returns:
        folium.Map: The generated map with markers and polygons representing the routes.
    """
    # Ensure dataset is sorted by 'NEW RT' and 'NEW DAY' for coherent route tracing
    dataset = dataset.sort_values(['NEW RT', 'NEW DAY'])
    
    # Initialize map
    m = folium.Map(
        location=[39.7, -76.5],
        zoom_start=9,
        max_zoom=20,
        tiles='https://api.mapbox.com/styles/v1/mapbox/satellite-streets-v12/tiles/256/{z}/{x}/{y}@2x?access_token=pk.eyJ1Ijoic2FtYm9zaXMiLCJhIjoiY2xyenJyM2h5MWlvMDJpcGV3NnJ3MTdkeSJ9.AGWk1gj--GW99AM0IbF3rg',
        attr='Mapbox'
    )
    
    # Generate color map for routes
    vehicles = dataset['NEW RT'].unique()
    colors = list(mcolors.CSS4_COLORS.keys())[13:13+len(vehicles)]  # Adjust if more vehicles than colors
    color_map = dict(zip(vehicles, colors))
    
    for vehicle in vehicles:
        route = folium.FeatureGroup(name=f'Route {vehicle}')
        vehicle_data = dataset[dataset['NEW RT'] == vehicle]
        
        # Add points and lines for each route
        for index, row in vehicle_data.iterrows():
            # Tooltip and popup text
            tooltip_text = f"Route: {vehicle}, Day: {row['NEW DAY']}, Sales: ${row['Machine']:,.2f}"
            folium.CircleMarker(
                location=[row['Latitude'], row['Longitude']],
                tooltip=tooltip_text,
                color=color_map[vehicle],
                fill=True,
                fill_color=color_map[vehicle],
                radius=5
            ).add_to(route)
        
        # Add route lines
        locations = vehicle_data[['Latitude', 'Longitude']].values.tolist()
        if len(locations) > 1:  # Only draw lines if more than one location
            folium.PolyLine(locations=locations, color=color_map[vehicle], weight=2.5, opacity=0.8).add_to(route)
        
        route.add_to(m)
    
    # Add fullscreen button
    Fullscreen().add_to(m)
    
    # Add LayerControl
    folium.LayerControl().add_to(m)
    
    # Save the map
    m.save("clean_map.html")
    return m

def create_map_with_convex_hull(dataset):
    """
    Generates a map with markers, lines, and convex hull polygons to visualize routes based on the 'NEW RT' column.

    Args:
        dataset (DataFrame): A pandas DataFrame containing latitude, longitude, and other data for each point.

    Returns:
        folium.Map: The generated map with markers, lines, and convex hull polygons representing the routes.
    """
    dataset = dataset.sort_values(['NEW RT', 'NEW DAY'])
    
    m = folium.Map(
        location=[39.7, -76.5],
        zoom_start=9,
        max_zoom=20,
        tiles='https://api.mapbox.com/styles/v1/mapbox/navigation-day-v1/tiles/256/{z}/{x}/{y}@2x?access_token=pk.eyJ1Ijoic2FtYm9zaXMiLCJhIjoiY2xyenJyM2h5MWlvMDJpcGV3NnJ3MTdkeSJ9.AGWk1gj--GW99AM0IbF3rg',
        attr='Mapbox'
    )
    
    vehicles = dataset['NEW RT'].unique()
    colors = list(mcolors.CSS4_COLORS.keys())[13:13+len(vehicles)]  # Adjust if more vehicles than colors
    color_map = dict(zip(vehicles, colors))
    
    for vehicle in vehicles:
        route = folium.FeatureGroup(name=f'Route {vehicle}')
        vehicle_data = dataset[dataset['NEW RT'] == vehicle]
        
        # Add points for each vehicle
        for index, row in vehicle_data.iterrows():
            folium.CircleMarker(
                location=[row['Latitude'], row['Longitude']],
                color=color_map[vehicle],
                fill=True,
                fill_color=color_map[vehicle],
                radius=5
            ).add_to(route)
        
        locations = vehicle_data[['Latitude', 'Longitude']].values
        if len(locations) > 2:  # ConvexHull requires at least 3 points
            try:
                hull = ConvexHull(locations)
                # Drawing Convex Hull
                hull_points = locations[hull.vertices]
                folium.Polygon(
                    locations=hull_points.tolist(),
                    color=color_map[vehicle],
                    fill=True,
                    fill_color=color_map[vehicle],
                    fill_opacity=0.3,
                    weight=2
                ).add_to(route)
            except Exception as e:
                print(f"Could not create a convex hull for vehicle {vehicle}: {e}")
                
        route.add_to(m)
    
    Fullscreen().add_to(m)
    folium.LayerControl().add_to(m)
    
    m.save("map_with_convex_hull.html")
    return m

def create_map_with_convex_hull_and_tooltip(dataset):
    """
    Generates a map with markers, lines, convex hull polygons, and tooltips to visualize routes based on 'NEW RT'.

    Args:
        dataset (DataFrame): A pandas DataFrame containing latitude, longitude, and other data for each point.

    Returns:
        folium.Map: The generated map with markers, lines, convex hull polygons, and tooltips representing the routes.
    """
    dataset = dataset.sort_values(['NEW RT', 'NEW DAY'])
    m = folium.Map(
        location=[39.7, -76.5],
        zoom_start=9,
        max_zoom=20,
        tiles='https://api.mapbox.com/styles/v1/mapbox/navigation-day-v1/tiles/256/{z}/{x}/{y}@2x?access_token=pk.eyJ1Ijoic2FtYm9zaXMiLCJhIjoiY2xyenJyM2h5MWlvMDJpcGV3NnJ3MTdkeSJ9.AGWk1gj--GW99AM0IbF3rg',
        attr='Mapbox'
    )
    vehicles = dataset['NEW RT'].unique()
    colors = list(mcolors.CSS4_COLORS.keys())[13:13+len(vehicles)]  # Adjust if more vehicles than colors
    color_map = dict(zip(vehicles, colors))
    
    for vehicle in vehicles:
        route = folium.FeatureGroup(name=f'Route {vehicle}')
        vehicle_data = dataset[dataset['NEW RT'] == vehicle]
        
        # Add points with tooltips for each vehicle
        for index, row in vehicle_data.iterrows():
            tooltip_html = f"""
            Route: {vehicle}, Day: {row['NEW DAY']}<br>
            Customer: {row['CUST NAME']}<br>
            Sales: ${row['Machine']:,.2f}
            """
            folium.CircleMarker(
                location=[row['Latitude'], row['Longitude']],
                tooltip=tooltip_html,
                color=color_map[vehicle],
                fill=True,
                fill_color=color_map[vehicle],
                radius=5
            ).add_to(route)
        
        locations = vehicle_data[['Latitude', 'Longitude']].values
        if len(locations) > 2:  # ConvexHull requires at least 3 points
            try:
                hull = ConvexHull(locations)
                hull_points = locations[hull.vertices]
                folium.Polygon(
                    locations=hull_points.tolist(),
                    color=color_map[vehicle],
                    fill=True,
                    fill_color=color_map[vehicle],
                    fill_opacity=0.3,
                    weight=2
                ).add_to(route)
            except Exception as e:
                print(f"Could not create a convex hull for vehicle {vehicle}: {e}")
                
        route.add_to(m)
    
    Fullscreen().add_to(m)
    folium.LayerControl().add_to(m)
    
    m.save("map_with_convex_hull_and_tooltip.html")
    return m
from folium.plugins import Fullscreen

def create_map_with_accessible_tooltips(dataset):
    """
    Generates a map ensuring markers are on top of polygons for accessible tooltips.

    Args:
        dataset (DataFrame): DataFrame containing latitude, longitude, and other data for each point.

    Returns:
        folium.Map: Generated map with markers on top of polygons.
    """
    dataset = dataset.sort_values(['NEW RT', 'NEW DAY'])
    
    m = folium.Map(
        location=[39.7, -76.5],
        zoom_start=9,
        max_zoom=20,
        tiles='https://api.mapbox.com/styles/v1/mapbox/navigation-day-v1/tiles/256/{z}/{x}/{y}?access_token=pk.eyJ1Ijoic2FtYm9zaXMiLCJhIjoiY2xyenJyM2h5MWlvMDJpcGV3NnJ3MTdkeSJ9.AGWk1gj--GW99AM0IbF3rg',
        attr='Mapbox'
    )
    
    vehicles = dataset['NEW RT'].unique()
    colors = list(mcolors.CSS4_COLORS.keys())[13:13+len(vehicles)]
    color_map = dict(zip(vehicles, colors))
    
    marker_groups = {}
    
    # Create polygons first
    for vehicle in vehicles:
        vehicle_data = dataset[dataset['NEW RT'] == vehicle]
        locations = vehicle_data[['Latitude', 'Longitude']].values
        
        if len(locations) > 2:
            try:
                hull = ConvexHull(locations)
                hull_points = locations[hull.vertices]
                folium.Polygon(
                    locations=hull_points.tolist(),
                    color=color_map[vehicle],
                    fill=True,
                    fill_color=color_map[vehicle],
                    fill_opacity=0.3,
                    weight=2
                ).add_to(m)
            except Exception as e:
                print(f"Could not create a convex hull for vehicle {vehicle}: {e}")
        
        # Prepare marker groups without adding them yet
        marker_groups[vehicle] = FeatureGroup(name=f'Route {vehicle}')
    
    # Add markers after polygons
    for vehicle, group in marker_groups.items():
        vehicle_data = dataset[dataset['NEW RT'] == vehicle]
        for index, row in vehicle_data.iterrows():
            tooltip_html = f"""
            Route: {vehicle}, Day: {row['NEW DAY']}<br>
            Customer: {row['CUST NAME']}<br>
            Address: {row['ADDRESS']}<br>
            Sales: ${row['Machine']:,.2f}<br>
            Companion: ${row['Companion']:,.2f}
            
            """
            folium.CircleMarker(
                location=[row['Latitude'], row['Longitude']],
                tooltip=tooltip_html,
                color=color_map[vehicle],
                fill=True,
                fill_color=color_map[vehicle],
                radius=5
            ).add_to(group)
        group.add_to(m)
    
    Fullscreen().add_to(m)
    folium.LayerControl().add_to(m)
    
    m.save("map_with_accessible_tooltips.html")
    return m


/tmp/ipykernel_1107483/2977106357.py:29: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [4]:
import pandas as pd
import numpy as np
file_path = "./rero.xls"
# load reroute.xls into dataset, set the column headers to the first row of the dataset and drop the second row
dataset = pd.read_excel(file_path)
dataset = dataset.iloc[1:, 0:]
dataset = dataset.reset_index(drop=True)
# if dataset does not have a column named 'Latitude' then create it
if 'Latitude' not in dataset.columns:
    dataset['Latitude'] = np.nan
# if dataset does not have a column named 'Longitude' then create it  
if 'Longitude' not in dataset.columns:
    dataset['Longitude'] = np.nan
#rename BIG 5 AVG to Machine and COMP AVG to Companion
#dataset = dataset.rename(columns={'BIG 5 AVG': 'Machine', 'COMP AVG': 'Companion'})
prev_obj, best_obj = 999999999, 999999999
try:
    dataset['Machine'] =dataset['BIG 5 AVG'] 
    dataset['Companion']=dataset['COMP AVG'] 
except Exception as e:
    print(e)
dataset['DAY'] = dataset['DAY'].str[:2].astype(int)
dataset['RT'] = dataset['RT'].astype(int)
# split the 'ADDRESS' column by comma and keep only the first part
dataset['ADDRESS'] = dataset['ADDRESS'].str.split(',').str[0]
import os
from tqdm import tqdm

if os.path.exists('coordinates.csv'):
    coordinates_df = pd.read_csv('coordinates.csv')
else:
    print("Creating new coordinates DataFrame...")
    coordinates_df = pd.DataFrame(columns=['CUST', 'Latitude', 'Longitude'])
dataset['CUST'] = dataset['CUST'].astype(int)
coordinates_df['CUST'] = coordinates_df['CUST'].astype(int)


geolocator = Nominatim(user_agent="my-stuff", timeout=10)
for i in tqdm(range(len(dataset))):
    #clear search_address
    search_address = ''
    cust = dataset.loc[i, 'CUST']
    # First, try to get coordinates from coordinates_df
    coord_row = coordinates_df[coordinates_df['CUST'] == cust]
    if not coord_row.empty:
        dataset.loc[i, 'Latitude'] = coord_row.iloc[0]['Latitude']
        dataset.loc[i, 'Longitude'] = coord_row.iloc[0]['Longitude']
        continue  # Skip to next iteration

    short_zip = str(dataset['ZIP'][i])
    search_address = f"{dataset['CUST NAME'][i]}, {dataset['HOUSE NUMBER'][i]} {dataset['ADDRESS'][i]}, {dataset['CITY'][i]}, {short_zip[:5]}"
    location = geolocator.geocode(search_address)   # geocode method returns an object of type location
    if location is not None:
        dataset.loc[i, 'Latitude'] = location.latitude
        dataset.loc[i, 'Longitude'] = location.longitude
    else:
        search_address = f"{dataset['CITY'][i]} {short_zip[:5]} "
        location = geolocator.geocode(search_address)   # geocode method returns an object of type location
        if location is not None:
            dataset.loc[i, 'Latitude'] = location.latitude
            dataset.loc[i, 'Longitude'] = location.longitude
        else:
            search_address = short_zip[:5]               
            if location is not None:
                dataset.loc[i, 'Latitude'] = location.latitude
                dataset.loc[i, 'Longitude'] = location.longitude
    new_row = pd.DataFrame({'CUST': [cust], 'Latitude': [location.latitude], 'Longitude': [location.longitude]})
    coordinates_df = pd.concat([coordinates_df, new_row], ignore_index=True)
#save dataset to csv
coordinates_df.to_csv('coordinates.csv', index=False)
def clean_data(dataset):
    # Ensure 'CUST' column is of consistent data type
    # Find the rows where the customer is the one in question and replace their coordinates
    dataset.loc[dataset['CUST'] == 216500035, 'Longitude'] = -76.376
    dataset.loc[dataset['CUST'] == 216500035, 'Latitude'] = 39.511
    return dataset
dataset = clean_data(dataset.copy())

  0%|          | 0/376 [00:00<?, ?it/s]

100%|██████████| 376/376 [00:00<00:00, 1465.54it/s]


In [5]:
#create a new distance matrix with the new latitudes and longitudes
new_distance_matrix = pd.DataFrame(index=dataset.index, columns=dataset.index)
new_distance_matrix.head()
for i in range(len(dataset)):
    for j in range(len(dataset)):
        new_distance_matrix.iloc[i, j] = haversinedm(dataset['Latitude'][i], dataset['Longitude'][i], dataset['Latitude'][j], dataset['Longitude'][j])
distance_matrix = new_distance_matrix
#distance_matrix = distance_matrix.astype(int)
distance_matrix = distance_matrix.add(.9999)
distance_matrix = distance_matrix.astype(int)
latlong_df = dataset[['Latitude', 'Longitude']]
distance_matrix.to_csv('distance_matrix.csv', index=False)

In [6]:
import pandas as pd
import hygese

In [10]:
def create_data_model(dataset, distance_matrix, vehicle_capacity):
    data = dict()
    demands = dataset['Companion'].tolist()
    data['distance_matrix'] = distance_matrix.values.tolist()
    data['num_vehicles'] = 3
    data['depot'] = 0
    data['demands'] = [int(demand) for demand in demands]
    data['vehicle_capacity'] = vehicle_capacity
    data['service_times'] = [0] * len(data['demands'])
    return data

In [11]:

## This is the main code that will be used to solve the problem and display the results
#display(compute_time)
vehicle_capacity = 10500
compute_time = 30
data = create_data_model(dataset, distance_matrix, vehicle_capacity)

In [12]:
import numpy as np 
import hygese as hgs 

# A CVRP from https://developers.google.com/optimization/routing/cvrp


# Solver initialization
ap = hgs.AlgorithmParameters(timeLimit=5.2)  # seconds
hgs_solver = hgs.Solver(parameters=ap, verbose=True)
print("solver Initialized")
# Solve
result = hgs_solver.solve_cvrp(data)
print("Done")
print(result.cost)
print(result.routes)

----- FLEET SIZE SPECIFIED: SET TO 3 VEHICLES
----- INSTANCE SUCCESSFULLY LOADED WITH 375 CLIENTS AND 3 VEHICLES
----- BUILDING INITIAL POPULATION
----- STARTING GENETIC ALGORITHM
It      0      2 | T(s) 1.35 | Feas 60 1192.00 1217.96 | NO-INFEASIBLE | Div 0.51 -1.00 | Feas 1.00 1.00 | Pen 0.11 0.85


In [2]:
import numpy as np
from hygese import AlgorithmParameters, Solver
def get_data():
    data = dict()
    data['distance_matrix'] = [
        [0, 548, 776, 696, 582, 274, 502, 194, 308, 194, 536, 502, 388, 354, 468, 776, 662],
        [548, 0, 684, 308, 194, 502, 730, 354, 696, 742, 1084, 594, 480, 674, 1016, 868, 1210],
        [776, 684, 0, 992, 878, 502, 274, 810, 468, 742, 400, 1278, 1164, 1130, 788, 1552, 754],
        [696, 308, 992, 0, 114, 650, 878, 502, 844, 890, 1232, 514, 628, 822, 1164, 560, 1358],
        [582, 194, 878, 114, 0, 536, 764, 388, 730, 776, 1118, 400, 514, 708, 1050, 674, 1244],
        [274, 502, 502, 650, 536, 0, 228, 308, 194, 240, 582, 776, 662, 628, 514, 1050, 708],
        [502, 730, 274, 878, 764, 228, 0, 536, 194, 468, 354, 1004, 890, 856, 514, 1278, 480],
        [194, 354, 810, 502, 388, 308, 536, 0, 342, 388, 730, 468, 354, 320, 662, 742, 856],
        [308, 696, 468, 844, 730, 194, 194, 342, 0, 274, 388, 810, 696, 662, 320, 1084, 514],
        [194, 742, 742, 890, 776, 240, 468, 388, 274, 0, 342, 536, 422, 388, 274, 810, 468],
        [536, 1084, 400, 1232, 1118, 582, 354, 730, 388, 342, 0, 878, 764, 730, 388, 1152, 354],
        [502, 594, 1278, 514, 400, 776, 1004, 468, 810, 536, 878, 0, 114, 308, 650, 274, 844],
        [388, 480, 1164, 628, 514, 662, 890, 354, 696, 422, 764, 114, 0, 194, 536, 388, 730],
        [354, 674, 1130, 822, 708, 628, 856, 320, 662, 388, 730, 308, 194, 0, 342, 422, 536],
        [468, 1016, 788, 1164, 1050, 514, 514, 662, 320, 274, 388, 650, 536, 342, 0, 764, 194],
        [776, 868, 1552, 560, 674, 1050, 1278, 742, 1084, 810, 1152, 274, 388, 422, 764, 0, 798],
        [662, 1210, 754, 1358, 1244, 708, 480, 856, 514, 468, 354, 844, 730, 536, 194, 798, 0]
    ]
    data['num_vehicles'] = 4
    data['depot'] = 0
    data['demands'] = [0, 1, 1, 2, 4, 2, 4, 8, 8, 1, 2, 1, 2, 4, 4, 8, 8]
    data['vehicle_capacity'] = 15  # different from OR-Tools: homogeneous capacity
    print(data['distance_matrix'])
    return data


def test_cvrp():
    data = get_data()

    # Solver initialization
    ap = AlgorithmParameters()
    ap.timeLimit = 1.1
    hgs_solver = Solver(ap, True)

    # Solve
    result = hgs_solver.solve_cvrp(data)
    print(result.cost)
    print(result.routes)
    assert (result.cost == 6208)


def test_cvrp_inputs():
    data = get_data()

    # Solver initialization
    ap = AlgorithmParameters(timeLimit=1.1)
    hgs_solver = Solver(parameters=ap, verbose=True)

    # Solve
    result = hgs_solver.solve_cvrp(data)
    print(result.cost)
    print(result.routes)
    assert (result.cost == 6208)


def test_cvrp_dist_mtx():
    # Solver initialization
    ap = AlgorithmParameters(timeLimit=3.1)
    hgs_solver = Solver(parameters=ap, verbose=True)

    data = dict()
    n = 17
    x = np.random.rand(n) * 1000
    y = np.random.rand(n) * 1000
    data['x_coordinates'] = x
    data['y_coordinates'] = y

    data['depot'] = 0
    data['demands'] = [0, 1, 1, 2, 4, 2, 4, 8, 8, 1, 2, 1, 2, 4, 4, 8, 8]
    data['vehicle_capacity'] = 15  # different from OR-Tools: homogeneous capacity

    # Solve with calculated distances
    dist_mtx = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            dist_mtx[i][j] = np.sqrt(
                np.square(x[i] - x[j]) + np.square(y[i] - y[j])
            )
    data['distance_matrix'] = dist_mtx
    result1 = hgs_solver.solve_cvrp(data)

    # solve without distance_matrix
    data.pop("distance_matrix", None)
    result2 = hgs_solver.solve_cvrp(data, rounding=False)
    assert abs(result1.cost - result2.cost) < 1e-3


def test_cvrp_duration():
    n = 10
    data = dict()
    data['x_coordinates'] = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
    data['y_coordinates'] = [5, 4, 3, 2, 1, 9, 8, 7, 6, 5]
    data['demands'] = [0, 2, 3, 1, 2, 3, 1, 2, 3, 1]
    data['vehicle_capacity'] = 10
    data['duration_limit'] = 18
    data['num_vehicles'] = 5

    # Solver initialization
    ap = AlgorithmParameters(timeLimit=1.1, seed=12, useSwapStar=True)
    hgs_solver = Solver(parameters=ap, verbose=True)

    result = hgs_solver.solve_cvrp(data, rounding=True)
    assert result.cost == 42

In [9]:
#create_data_model(dataset, distance_matrix, vehicle_capacity)
#get_data()
#test_cvrp_inputs()
#test_cvrp()
#test_cvrp_dist_mtx()
#test_cvrp_duration()

NameError: name 'create_data_model' is not defined

In [10]:
# A CVRP from https://developers.google.com/optimization/routing/cvrp
import numpy as np 
import hygese as hgs 

data = dict()
data['distance_matrix'] = [
    [0, 548, 776, 696, 582, 274, 502, 194, 308, 194, 536, 502, 388, 354, 468, 776, 662],
    [548, 0, 684, 308, 194, 502, 730, 354, 696, 742, 1084, 594, 480, 674, 1016, 868, 1210],
    [776, 684, 0, 992, 878, 502, 274, 810, 468, 742, 400, 1278, 1164, 1130, 788, 1552, 754],
    [696, 308, 992, 0, 114, 650, 878, 502, 844, 890, 1232, 514, 628, 822, 1164, 560, 1358],
    [582, 194, 878, 114, 0, 536, 764, 388, 730, 776, 1118, 400, 514, 708, 1050, 674, 1244],
    [274, 502, 502, 650, 536, 0, 228, 308, 194, 240, 582, 776, 662, 628, 514, 1050, 708],
    [502, 730, 274, 878, 764, 228, 0, 536, 194, 468, 354, 1004, 890, 856, 514, 1278, 480],
    [194, 354, 810, 502, 388, 308, 536, 0, 342, 388, 730, 468, 354, 320, 662, 742, 856],
    [308, 696, 468, 844, 730, 194, 194, 342, 0, 274, 388, 810, 696, 662, 320, 1084, 514],
    [194, 742, 742, 890, 776, 240, 468, 388, 274, 0, 342, 536, 422, 388, 274, 810, 468],
    [536, 1084, 400, 1232, 1118, 582, 354, 730, 388, 342, 0, 878, 764, 730, 388, 1152, 354],
    [502, 594, 1278, 514, 400, 776, 1004, 468, 810, 536, 878, 0, 114, 308, 650, 274, 844],
    [388, 480, 1164, 628, 514, 662, 890, 354, 696, 422, 764, 114, 0, 194, 536, 388, 730],
    [354, 674, 1130, 822, 708, 628, 856, 320, 662, 388, 730, 308, 194, 0, 342, 422, 536],
    [468, 1016, 788, 1164, 1050, 514, 514, 662, 320, 274, 388, 650, 536, 342, 0, 764, 194],
    [776, 868, 1552, 560, 674, 1050, 1278, 742, 1084, 810, 1152, 274, 388, 422, 764, 0, 798],
    [662, 1210, 754, 1358, 1244, 708, 480, 856, 514, 468, 354, 844, 730, 536, 194, 798, 0]
]
data['num_vehicles'] = 4
data['depot'] = 0
data['demands'] = [0, 1, 1, 2, 4, 2, 4, 8, 8, 1, 2, 1, 2, 4, 4, 8, 8]
data['vehicle_capacity'] = 15  # different from OR-Tools: homogeneous capacity
data['service_times'] = np.zeros(len(data['demands']))

# Solver initialization
ap = hgs.AlgorithmParameters(timeLimit=3.2)  # seconds
hgs_solver = hgs.Solver(parameters=ap, verbose=True)

# Solve
result = hgs_solver.solve_cvrp(data)
print(result.cost)
print(result.routes)

----- FLEET SIZE SPECIFIED: SET TO 4 VEHICLES
----- INSTANCE SUCCESSFULLY LOADED WITH 16 CLIENTS AND 4 VEHICLES
----- BUILDING INITIAL POPULATION
----- STARTING GENETIC ALGORITHM
It      0      2 | T(s) 0.01 | Feas 59 6208.00 6208.00 | Inf 1 7696.90 7696.90 | Div 0.12 -nan | Feas 0.99 1.00 | Pen 164.90 0.85
It    500    502 | T(s) 0.05 | Feas 65 6208.00 6208.00 | Inf 4 6516.17 7117.67 | Div 0.12 0.41 | Feas 0.98 1.00 | Pen 73.17 0.38
It   1000   1002 | T(s) 0.11 | Feas 49 6208.00 6208.00 | Inf 25 5677.94 6127.94 | Div 0.15 0.39 | Feas 0.30 1.00 | Pen 32.46 0.17
It   1500   1502 | T(s) 0.18 | Feas 52 6208.00 6208.00 | Inf 55 5697.42 5835.48 | Div 0.13 0.15 | Feas 0.26 1.00 | Pen 33.11 0.10
It   2000   2002 | T(s) 0.24 | Feas 40 6208.00 6208.00 | Inf 47 5717.28 5886.10 | Div 0.17 0.20 | Feas 0.25 1.00 | Pen 33.78 0.10
It   2500   2502 | T(s) 0.31 | Feas 53 6208.00 6208.00 | Inf 57 5758.22 5893.15 | Div 0.13 0.16 | Feas 0.04 1.00 | Pen 35.14 0.10
It   3000   3002 | T(s) 0.38 | Feas 63 620